In [67]:
class Foo:
    
    def __init__(self):
        self._it = list(range(0, 30, 2))
    
    def __getitem__(self, pos):
        return self._it[pos]
    
    def __iter__(self):
        return iter(self._it)
    
    def __contains__(self, value):
        return value in self._it
    
    def __len__(self):
        return len(self._it)
    
    def __setitem__(self, pos, value):  # mutable sequence
        self._it[pos] = value
    
f = Foo()

In [70]:
for i in f: print(i)

22
24
28
14
6
26
16
4
0
18
10
2
8
12
20


In [71]:
20 in f

True

In [72]:
15 in f

False

In [69]:
from random import shuffle
shuffle(f)

In [65]:
f

### Subclassing an ABC  

In [73]:
#frenchDeck2
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, value):
        self._cards.insert(position, value)
        
poke = FrenchDeck2()

/Users/linheng/dev/py37/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


In [76]:
for card in poke:
    print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')
Card(rank='7', suit='spades')
Card(rank='8', suit='spades')
Card(rank='9', suit='spades')
Card(rank='10', suit='spades')
Card(rank='J', suit='spades')
Card(rank='Q', suit='spades')
Card(rank='K', suit='spades')
Card(rank='A', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='3', suit='diamonds')
Card(rank='4', suit='diamonds')
Card(rank='5', suit='diamonds')
Card(rank='6', suit='diamonds')
Card(rank='7', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='A', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='8', sui

## Defining and using an ABC  

The Tombola ABC has four methods.  
* load():  abstract methods
* picd():  abstract methods
* loaded():  return True if there is one item at least
* inspect():  return a sorted *tuple* built from the items

### Python2  
class Tombola(object):  
    \_\_metaclass\_\_ = abc.ABCMeta  

### Python3.3 (<3.4)  
class Tombola(metaclass=abc.ABCMeta):  
    #...  

### Python3.7 (>=3.4)
class Tombola(abc.ABC):  
    #...  
            

In [9]:
# tombola.py
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable"""
        
    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        This method should raise 'LookupError' when the instance is empty.
        """
        
    def loaded(self):
        """Return 'True' if there's at lease 1 item, 'False' otherwise"""
        return bool(self.inspect())
    
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [14]:
# A fake Tombola subclass doesn't go
from tombola import Tombola
class Fake(Tombola):
    def pick(self):
        return 13

type(Fake)

__main__.Fake

In [15]:
Fake

__main__.Fake

In [16]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [17]:
#The preferred way to declare an abstract class method
class MyABC(abc.ABC): 
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, ...):
        pass

SyntaxError: invalid syntax (<ipython-input-17-f414b0ae2c5a>, line 5)

### Subclassing the Tombola ABC

In [18]:
# bingo.py
import random

from tombola import Tombola

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

In [19]:
bc = BingoCage(range(1, 20, 2))

In [20]:
bc

In [21]:
bc.pick()

19

In [24]:
bc()

In [25]:
bc.inspect()

(1, 3, 5, 7, 11, 13, 17)

In [26]:
bc.loaded()

True

In [27]:
bc()

In [28]:
bc.inspect()

(1, 3, 5, 11, 13, 17)